##### Date : 10/31/2022
##### Author : Meriem HAFID

In [33]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy.linalg as linalg
plt.rcParams["figure.figsize"] = (16,9)

#### The boundary  condition for a Call spread : 
First For a Call option, if the asset is worthless there’s no point on buying it and therefore on exercising the option, so the option itself is worthless. On the other hand, if the value of the asset is very high, we will exercise it very surely in the expiry day, since we know that we will pay K for it in the future, the payment on present money may be estimated as $Ke^{−r(T −t)}$ and the final profit, and therefore the value of the option would be $S_{max} − Ke^{−r(T −t)}$, we can conclude on spread call for a combination of a short leg - call and long leg - call its value would be $(S_{max} − K_{long}e^{−r(T −t)}) - (S_{max} − K_{short}e^{−r(T −t)})$

Boundary conditions :

$S→0$ is known and is equal to 0.

$S→∞$ $f(t,S)=(K_{short}−K_{long})e^{−r(T−t)}$

#### The boundary  condition for a put : 
For a Put option, if the asset is worthless, it’s very surely that we will decide to exercise the option, getting a future profit of K, so the estimation of the present value can be said to be $f(0, t)$, on the other hand, if the asset is very expensive, probably we won’t exercise it, because it’s more profitable to sell directly in the market, so the option is worthless, thus $f(S_{max}, 0) = 0$

$f(S, T) = (K − S)^{+} = max(K − S, 0)$ (Terminal condition)

$f(0, t) = Ke^{−r(T −t)}, f(S_{max}, t) = 0$ (Boundary conditions)

## Implicit scheme : 

In [307]:
def pde_scheme_implicit(S_0, r, sigma, T, K_long, K_short, H, nb_x_side, nb_t,is_call_spread):
    nb_x = 2 * nb_x_side + 1
    xs = np.linspace(np.log(S_0)-H, np.log(S_0)+H, nb_x)
    dx = 2. * H / (nb_x - 1.) 
    dt = T / (nb_t - 1.)
    ts = np.linspace(0, T, nb_t)
    p = np.empty([nb_x, nb_t])
    
    #Boundary condition on the call spread
    if is_call_spread==True:
        g = lambda S : np.maximum(S-K_long,0.) - np.maximum(S-K_short,0.)
        p[:,0] = g(np.exp(xs))
        p[0,:] = 0.
        p[-1,:] = (K_short - K_long) * np.exp(-r*ts)
    
    #Boundary condition on the put
    else:
        g = lambda S : np.maximum(K_long-S,0.)
        p[:,0] = g(np.exp(xs))
        p[0,:] = K_long* np.exp(-r*ts) 
        p[-1,:] = 0.
        
    d = 1.+dt*(r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2)
    sup_d = -dt*((r-0.5*sigma**2)/dx + 0.5 * sigma**2 / dx**2)
    inf_d = -dt*(0.5 * sigma**2 / dx**2)
    A = np.diag(d * np.ones(nb_x-2)) + np.diag(sup_d * np.ones(nb_x-3), 1) + np.diag(inf_d * np.ones(nb_x-3), -1)
    v = np.zeros_like(p[1:-1,0])
    v[-1] = 1.
    invA = np.linalg.inv(A)
    for t in range(1,nb_t):
        p[1:-1,t] = invA @ (p[1:-1,t-1] - sup_d * p[-1,t] * v)   
    return p, p[nb_x_side,-1]

In [286]:
def pde_scheme_implicit_neumann(S_0, r, sigma, T, g, H, nb_x_side, nb_t):
    nb_x = 2 * nb_x_side + 1
    xs = np.linspace(np.log(S_0)-H, np.log(S_0)+H, nb_x)
    dx = 2. * H / (nb_x - 1.) 
    dt = T / (nb_t - 1.)
    ts = np.linspace(0, T, nb_t)
    p = np.empty([nb_x, nb_t])
    p[:,0] = g(np.exp(xs))
    d = 1.+dt*(r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2)
    sup_d = -dt*((r-0.5*sigma**2)/dx + 0.5 * sigma**2 / dx**2)
    inf_d = -dt*(0.5 * sigma**2 / dx**2)
    A = np.diag(d * np.ones(nb_x)) + np.diag(sup_d * np.ones(nb_x-1), 1) + np.diag(inf_d * np.ones(nb_x-1), -1)
    A[0,0] = 1.+dt*(r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2 / 2.)
    A[-1,-1] = 1.+dt*(r + sigma**2 / dx**2 / 2.)
    invA = np.linalg.inv(A)
    for t in range(1,nb_t):
        p[:,t] = invA @ p[:,t-1]  
    return p, p[nb_x_side,-1]

## Crank-Nicolson Scheme : 
We first set the initial condition and boundary condition, and calculate the $\alpha$, $\beta$, $\gamma$ coefficient. 

In [273]:
class FD_Crank_Nicolson(object):
    
    def __init__(self, S_0, K_long, K_short, r, T, sigma, S_max, nb_x_side, nb_t, is_put):
        self.S_0 = S_0
        self.K_long = K_long
        self.K_short = K_short
        self.r = r
        self.T = T
        self.sigma = sigma
        self.S_max = S_max
        self.nb_x_side, self.nb_t = nb_x_side, nb_t
        self.is_call_spread = not is_put
        self.i_values = np.arange(self.nb_x_side)
        self.j_values = np.arange(self.nb_t)
        self.p = np.zeros(shape=(self.nb_x_side+1, self.nb_t+1))
        self.boundary_conds = np.linspace(0, S_max, self.nb_x_side+1)
        self.dS = self.S_max/float(self.nb_x_side)
        self.dt = self.T/float(self.nb_t)
    
    def setup_boundary_conditions(self):
        if self.is_call_spread:
            self.p[:,-1] = np.maximum(self.boundary_conds - self.K_long,0) - np.maximum(self.boundary_conds - self.K_short,0)
            self.p[-1,:-1] = (self.K_short - self.K_long) * np.exp(-self.r*self.dt*(self.nb_t-self.j_values))
        else:
            self.p[:,-1] = np.maximum(0, self.K_long-self.boundary_conds)
            self.p[0,:-1] = (self.K_long-self.S_max) * np.exp(-self.r*self.dt*(self.nb_t-self.j_values))

    def setup_coefficients(self):
        self.alpha = 0.25*self.dt*((self.sigma**2)*(self.i_values**2) - self.r*self.i_values)
        self.beta = -self.dt*0.5*((self.sigma**2)*(self.i_values**2) + self.r)
        self.gamma = 0.25*self.dt*((self.sigma**2)*(self.i_values**2) + self.r*self.i_values)
        self.nb_x1 = -np.diag(self.alpha[2:self.nb_x_side], -1) + np.diag(1-self.beta[1:self.nb_x_side]) - np.diag(self.gamma[1:self.nb_x_side-1], 1)
        self.nb_x2 = np.diag(self.alpha[2:self.nb_x_side], -1) + np.diag(1+self.beta[1:self.nb_x_side]) + np.diag(self.gamma[1:self.nb_x_side-1], 1)

    def traverse_grid(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.nb_x1)

        for j in reversed(range(self.nb_t)):
            x1 = linalg.solve(L, np.dot(self.nb_x2, self.p[1:self.nb_x_side, j+1]))
            x2 = linalg.solve(U, x1)
            self.p[1:self.nb_x_side, j] = x2
            
    def price(self):
        self.setup_boundary_conditions()
        self.setup_coefficients()
        self.traverse_grid()
        
        """Use piecewise linear interpolation on the initial p column to get the closest price at S0."""
        return np.interp(self.S_0, self.boundary_conds, self.p[:,0])

## A Call-Spread : 

We assume $S_{0} = 10$, $r = 2%$ and $σ = 25%$

• Long leg: at-the-money call with 3-year maturity 

• Short leg: call of maturity equal to 3 years with moneyness 150%.

In [192]:
S0 = 10.
H = np.log(3*S0) - np.log(S0)
nb_x_side = 500
nb_t = 1000
S_min = S0 * np.exp(-H)
S_max = S0 * np.exp(H)
K_long = 10. #The stike price = the underlaying price cause we are ATM 
K_short = 15. #Moneyess de 150% m = K/S K = 1.5*10 = 15
r = 0.02
sigma = 0.25
T = 3.

In [262]:
table_CallSpread, price_CallSpread = pde_scheme_implicit(S0, r, sigma, T, K_long, K_short, H, nb_x_side, nb_t, True)
print('Using the implicit scheme: the price of the call-spread is around {}.'.format(round(price_CallSpread,4)))

Using the implicit scheme: the price of the call-spread is around 1.3581.


In [234]:
CallSpread = FD_Crank_Nicolson(S0, K_long, K_short, r, T, sigma, S_max, nb_x_side, nb_t, is_put=False)
print('Using Crank Nicolson scheme: the price of the call-spread is around {}.'.format(round(CallSpread.price(),4)))

Using Crank Nicolson scheme: the price of the call-spread is around 1.3155.


## Put option: 

• a 10-year maturity put option with moneyness 80%.

In [313]:
S0 = 10.
H = np.log(20*S0) - np.log(S0)
nb_x_side = 500
nb_t = 1000
S_min = S0 * np.exp(-H)
S_max = S0 * np.exp(H)
K_put = 8. #Moneyess de 80% m = K/S K = 0.8*10 = 8
k = 0
r = 0.02
sigma = 0.25
T_put = 10.

In [312]:
table_put, price_put = pde_scheme_implicit(S0, r, sigma, T_put, K_put, k, H, nb_x_side, nb_t, False)
print('Using the implicit scheme: the price of the put is around {}.'.format(round(price_put,4))) 

Using the implicit scheme: the price of the put is around 1.1431.


In [261]:
Put = FD_Crank_Nicolson(S0, K_put, k, r, T_put, sigma, S_max, nb_x_side, nb_t, is_put=True)
print('Using Crank Nicolson scheme: the price of the put is around {}.'.format(round(Put.price(),4))) 

Using Crank Nicolson scheme: the price of the put is around 1.1485.
